In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

import tkinter as tk
from tkinter import filedialog as fd

import collections

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

In [9]:
file_shp_avance = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - CATASTRO\Catastro\Cosecha2023.shp'
file_xlsx_avance = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2023\AVANCE SEMANAL COSECHA 2023\RENDIMIENTO AVANCE 230723.xlsx'

In [11]:
# lectura de hoja 1
xlsx_avance = pd.read_excel(file_xlsx_avance, sheet_name='Hoja1')

In [12]:
# filtra por el grupo de cosecha de interes
grupo = xlsx_avance[xlsx_avance['COD COS'] == 282]
# seleccion de columnas de interes
grupo_cols = grupo[['INS', 'GRUPO', 'CODIGO CAÑERO ', 'CAÑERO', 'TN COMPROMISO INDIVIDUAL', 'TN ENTREGA INDIVIDUAL']]
grupo_cols

,INS,GRUPO,CODIGO CAÑERO,CAÑERO,TN COMPROMISO INDIVIDUAL,TN ENTREGA INDIVIDUAL
227,79.0,7.0,173.0,ALVAREZ SEJAS BISMARCK,665.10,442.99
228,79.0,7.0,409.0,ALVAREZ SEJAS MELANIA,530.66,0.00
229,79.0,7.0,7519.0,LEON ALVAREZ REMY,740.75,0.00
230,79.0,7.0,8025.0,VARGAS ALVAREZ KEVIN,226.40,96.04
231,79.0,7.0,10305.0,PERALTA LOPEZ MAIDE,2147.35,379.00
232,79.0,7.0,13012.0,SALAZAR ARNEZ LIDER,2235.90,0.00
233,79.0,7.0,14556.0,VARGAS PANOZO EDWIN GLADIMIRO,92.95,0.00
234,79.0,7.0,41554.0,LEDEZMA SALAZAR RICHARD JIMMY,755.21,896.82
235,79.0,7.0,41827.0,SALAZAR PERALTA GABRIELA,6247.02,720.97
236,89.0,5.0,41919.0,SALAZAR SEJAS HECTOR DANIEL,580.60,0.00


In [177]:
# Crear el namedtuple. Supongamos que tu DataFrame tiene columnas 'A', 'B' y 'C'
MyTuple = collections.namedtuple('MyTuple', ['INS', 'GRUPO', 'CODIGO CAÑERO ', 'CAÑERO', 'TN COMPROMISO INDIVIDUAL', 'TN ENTREGA INDIVIDUAL'])

# Convertir el DataFrame en una lista de namedtuples
#list_of_tuples = [MyTuple(*row) for row in grupo_cols.itertuples(index=False)]

ValueError: Type names and field names must be valid identifiers: 'CODIGO CAÑERO '

In [151]:
# total para sumatoria de columnas numerias
totales_grupo = {
    'total_compromiso' : grupo['TN COMPROMISO INDIVIDUAL'].sum(),
    'total_entregado' : grupo['TN ENTREGA INDIVIDUAL'].sum()
}
totales_grupo

{'total_compromiso': 14773.01, 'total_entregado': 2535.82}

## PORCENTAJE DE AVANCE

In [152]:
# seleccion de la primera fila del grupo
data_grupo = dict(grupo.iloc[0])

In [153]:
porcen_avance = {
    'TN COMPROMISO GRUPAL' : data_grupo['TN COMPROMISO GRUPAL'],
    'TN ENTREGA GRUPAL': data_grupo['TN ENTREGA GRUPAL'],
    '% ENTREGA GRUPAL' : data_grupo['% ENTREGA GRUPAL'] * 100,
    'HA CANHA POR GRUPO' : data_grupo['HA CANHA POR GRUPO'],
    'HA COSECHA GRUPAL' : data_grupo['HA COSECHA GRUPAL'],
    '% COSECHA GRUPAL' : data_grupo['% COSECHA GRUPAL'] * 100,
    'DIFERENCIA' : data_grupo['DIFERENCIA'] * 100
}
porcen_avance

{'TN COMPROMISO GRUPAL': 14773.01,
 'TN ENTREGA GRUPAL': 2535.8199999999997,
 '% ENTREGA GRUPAL': 17.165222253284874,
 'HA CANHA POR GRUPO': 258.68,
 'HA COSECHA GRUPAL': 51.370000000000005,
 '% COSECHA GRUPAL': 19.858512447811968,
 'DIFERENCIA': -2.6932901945270955}

## PORCENTAJE COMPROMETIDO

In [154]:
porcen_compromiso = {
    'TN COMPROMISO GRUPAL' : data_grupo['TN COMPROMISO GRUPAL'],
    'TN ENTREGA GRUPAL': data_grupo['TN ENTREGA GRUPAL'],
    'COMPROMISO RESTANTE' : data_grupo['COMPROMISO RESTANTE'],
    'TN ESTIMADA APROX' : data_grupo['TN ESTIMADA APROX'],
    'DIFERENCIA COMPROMETIDO' : data_grupo['DIFERENCIA COMPROMETIDO']
}
porcen_compromiso

{'TN COMPROMISO GRUPAL': 14773.01,
 'TN ENTREGA GRUPAL': 2535.8199999999997,
 'COMPROMISO RESTANTE': 12237.19,
 'TN ESTIMADA APROX': 11025.142905392251,
 'DIFERENCIA COMPROMETIDO': 1212.0470946077494}

## PROPIEDADES

In [155]:
# copia de grupo
grupo_aux = grupo.copy()
# cambio de nombre de columna de codigo cañero
grupo_aux.rename(columns={'CODIGO CAÑERO ':'cod_ca'}, inplace=True)

In [156]:
# carga de datos de shp de avance
shp_avance = gpd.read_file(file_shp_avance)

In [157]:
# copia de datos de shp_avance
shp_avance_aux = shp_avance.copy()
# cambio de nombre de columna de cod_cañero
shp_avance_aux.rename(columns={'unidad_03':'cod_ca'}, inplace=True)

In [158]:
# nueva columna de area cosechada (dependiendo del estado)
shp_avance_aux['area_cosechada'] = shp_avance_aux.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
# nueva columna de area no cosechada (resta de area - area_cosechada)
shp_avance_aux['area_sin_cosechar'] = shp_avance_aux.apply(lambda row : row['area'] - row['area_cosechada'], axis=1)

In [159]:
# merge de los datos de shp de avance y grupo
propiedades_merge = pd.merge(shp_avance_aux, grupo_aux, on='cod_ca')

In [167]:
# agrupacion de datos, para totalizar las areas cosechadas y las que no
propiedades_total = propiedades_merge.groupby(['unidad_01', 'unidad_02'])[['area_cosechada', 'area_sin_cosechar']].sum()

In [168]:
propiedades_total = propiedades_total.reset_index()
propiedades_total

,unidad_01,unidad_02,area_cosechada,area_sin_cosechar
0,78.0,SINFORIANO_V_Y_VI,0.00,20.54
1,244.0,SAN_VICENTE--PERALTA,24.00,40.48
2,577.0,SAN_ANTONIO--IAG,0.00,77.03
3,593.0,SINFORIANO_V--LEON,0.00,14.55
4,594.0,SINFORIANO_III--LEON,0.00,2.88
5,1091.0,LAS_MAROTAS--GERARDO_SALAZAR,48.47,8.78
6,1559.0,SAN_MARTIN--LIDER_SALAZAR,7.67,20.83
7,1579.0,SALAZAR--SALAZAR_LIDER,0.00,12.74
